# Clasificador: Naive Bayes


Los clasificadores Naive Bayes son una familia de "clasificadores probabilísticos"  basados en la aplicación del teorema de Bayes con fuertes supuestos de independencia (naive) entre las características.

Los clasificadores Naive Bayes son altamente escalables, y requieren un número de parámetros lineales en el número de variables (características / predictores) en un problema de ML.

Los modelos Naive Bayes se conocen bajo una variedad de nombres, incluyendo Bayes simples y Bayes independientes, pero todos estos nombres hacen referencia al uso del teorema de Bayes en la regla de decisión del clasificador, pero el naive Bayes  se refier mas a la independencia entre caracteristicas.

El problema de  tutorial usaremos en este tutorial es el problema de la Diabetes de los indios Pima . Este problema se compone de 768 observaciones  médicas para las pacientes de los indios Pima. Los registros describen las mediciones instantáneas tomadas del paciente, como su edad, el número de veces embarazadas y el análisis de sangre. Todos los pacientes son mujeres mayores de 21 años. Todos los atributos son numéricos y sus unidades varían de un atributo a otro.

Cada registro tiene un valor de clase que indica si el paciente sufrió una aparición de diabetes dentro de los 5 años posteriores a la toma de las mediciones (1) o no (0).

In [0]:
# Cargar el Drive helper y mount
from google.colab import drive

# Link y Codigo de autorizacion.
drive.mount('/content/drive_all')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dataset = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv',header=None, names=None, index_col=None)
dataset.sample()


Drive already mounted at /content/drive_all; to attempt to forcibly remount, call drive.mount("/content/drive_all", force_remount=True).


,0,1,2,3,4,5,6,7,8
439,6,107,88,0,0,36.8,0.727,31,0


### Dividir los datos en un conjunto de datos en entrenamiento y test

In [0]:
import random
def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]



In [0]:
dataset2 = [[1], [2], [3], [4], [5]]
splitRatio = 0.75
train, test = splitDataset(dataset2, splitRatio)
print('Split {0} rows into train with {1} and test with {2}',format(len(dataset2)), train, test)

Split {0} rows into train with {1} and test with {2} 5 [[1], [5], [4]] [[2], [3]]


**2. Resumir datos**
El modelo de Naive Bayes se compone de un resumen de los datos en el conjunto de entrenamiento. Este resumen luego se usa para hacer predicciones. El resumen de los datos de entrenamiento recopilados implica la media y la desviación estándar para cada atributo, por valor de clase. Por ejemplo, si hay dos valores de clase y 7 atributos numéricos, entonces necesitamos una media y una desviación estándar para cada combinación de atributo (7) y valor de clase (2), es decir, 14 resúmenes de atributos.

Estos son necesarios cuando se realizan predicciones para calcular la probabilidad de valores de atributos específicos que pertenecen a cada valor de clase.

*** Datos separados por clase**
La primera tarea es separar las instancias del conjunto de datos de entrenamiento por valor de clase para que podamos calcular estadísticas para cada clase. Podemos hacerlo creando un mapa de cada valor de clase a una lista de instancias que pertenecen a esa clase y ordenando el conjunto de datos completo de las instancias en las listas apropiadas.

In [0]:
def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated



In [0]:
dataset3 = [[1,20,1], [2,21,0], [3,22,1]]
separated = separateByClass(dataset3)
#print('Separated instances: {0}').format(separated)

*** Calcular la media**
Necesitamos calcular la media y  la desviación estándar de cada atributo para un valor de clase .


In [0]:
import math
def mean(numbers):
	return sum(numbers)/float(len(numbers))

def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)


**Resumir conjunto de datos**
Para una lista dada de instancias (para un valor de clase) podemos calcular la media y la desviación estándar para cada atributo.

La función zip agrupa los valores para cada atributo a través de las instancias de datos en sus propias listas.

In [0]:
def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del summaries[-1]
	return summaries



In [0]:
dataset4 = [[1,20,0], [2,21,1], [3,22,0]]
summary = summarize(dataset4)
print('Attribute summaries: {0}',format(summary))

Attribute summaries: {0} [(2.0, 1.0), (21.0, 1.0)]


**Resumir atributos por clase**
Podemos juntarlo todo separando primero nuestro conjunto de datos de entrenamiento en instancias agrupadas por clase. Luego calcula los resúmenes para cada atributo.

In [0]:
def summarizeByClass(dataset):
	separated = separateByClass(dataset)
	summaries = {}
	for classValue, instances in separated.items():
		summaries[classValue] = summarize(instances)
	return summaries



In [0]:
import timeit


dataset4 = [[1,20,1], [2,21,0], [3,22,1], [4,22,0]]
summary = summarizeByClass(dataset4)

print('Summary by class value: {0}',format(summary))



Summary by class value: {0} {1: [(2.0, 1.4142135623730951), (21.0, 1.4142135623730951)], 0: [(3.0, 1.4142135623730951), (21.5, 0.7071067811865476)]}


**Hacer predicción**
Ahora estamos listos para hacer predicciones utilizando los resúmenes preparados a partir de nuestros datos de entrenamiento.





**Calcular la función de densidad de probabilidad gaussiana**
Podemos usar una función gaussiana para estimar la probabilidad de un valor de atributo dado, dada la media conocida y la desviación estándar para el atributo estimado a partir de los datos de entrenamiento.

Dado que los resúmenes de atributos se prepararon para cada atributo y valor de clase, el resultado es la probabilidad condicional de un valor de atributo dado dado un valor de clase. Estamos conectando nuestros detalles conocidos en Gaussian (valor de atributo, media y desviación estándar) y leyendo la probabilidad de que nuestro valor de atributo pertenezca a la clase.

En la función **CalculateProbability ()** calculamos primero el exponente, luego calculamos la división principal. Esto nos permite ajustar bien la ecuación en dos líneas.

In [0]:
import math
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent




In [0]:
x = 71.5
mean = 73
stdev = 6.2
probability = calculateProbability(x, mean, stdev)
print('Probability of belonging to this class: {0}',format(probability))


Probability of belonging to this class: {0} 0.06248965759370005


**Calcular probabilidades de clase**
Ahora que podemos calcular la probabilidad de que un atributo pertenezca a una clase, podemos combinar las probabilidades de todos los valores de atributo para una instancia de datos y obtener una probabilidad de que toda la instancia de datos pertenezca a la clase.

In [0]:

def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev)
	return probabilities


In [0]:
summaries = {0:[(1, 0.5)], 1:[(20, 5.0)]}
inputVector = [1.1, '?']
probabilities = calculateClassProbabilities(summaries, inputVector)
print('Probabilities for each class: {0}',format(probabilities))

Probabilities for each class: {0} {0: 0.7820853879509118, 1: 6.298736258150442e-05}


**Haz una predicción**
Ahora que podemos calcular la probabilidad de que una instancia de datos pertenezca a cada valor de clase, podemos buscar la mayor probabilidad y devolver la clase asociada.

In [0]:


def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel


In [0]:

summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
inputVector = [1.1, '?']
result = predict(summaries, inputVector)
print('Prediction: {0}',format(result))

Prediction: {0} A


# Hacer predicciones
Finalmente, podemos estimar la precisión del modelo haciendo predicciones para cada instancia de datos en nuestro conjunto de datos de prueba. Los getPredictions () van a hacer esto y devolver una lista de predicciones para cada instancia de prueba.

In [0]:


def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions



In [0]:

summaries = {'A':[(1, 0.5)], 'B':[(20, 5.0)]}
testSet = [[1.1, '?'], [19.1, '?']]
predictions = getPredictions(summaries, testSet)
print('Predictions: {0}',format(predictions))

Predictions: {0} ['A', 'B']


**Obtener precisión**
Las predicciones se pueden comparar con los valores de clase en el conjunto de datos de prueba y la precisión de la clasificación se puede calcular como una relación de precisión entre 0 y 100%. El getAccuracy () calculará esta relación de precisión.

In [0]:


def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0


Podemos probar la función getAccuracy () usando el código de ejemplo a continuación.

In [0]:

testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: {0}',format(accuracy))

Accuracy: {0} 66.66666666666666


## La precision no es tan buena como esperamos, que tendriamos que hacer o cambiar para mejorarla?

In [0]:
## escribir codigo